In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json

In [2]:
playtest5_data = pd.read_csv("2018-11-14_Playtest/anonymized_playtest5_data.csv", sep=";")
playtest6_data = pd.read_csv("2019-01-07_Playtest/anonymized_playtest6_data.csv", sep=";")
playtest7_data = pd.read_csv("2019-01-31_Playtest/anonymized_playtest7_data.csv", sep=";")

In [3]:
def get_shapes(puzzle):
    shapes = []
    if puzzle is None:
        return shapes
    for shape_data in puzzle.get("shapeData"):
        shapes.append(shape_data.get('shapeType'))
    return shapes    

In [4]:
puzzles = dict()
with open("2019-01-07_Playtest/StreamingAssets/config.json") as f:
    asset_config = json.load(f)

for puzzle_sets in asset_config.get("puzzleSets"):
    for puzzle_file in puzzle_sets.get("puzzles"):
        with open(f"2019-01-07_Playtest/StreamingAssets/{puzzle_file}.json") as f:
            puzzle_details = json.load(f)
            puzzles[puzzle_details.get("puzzleName")] = puzzle_details

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
#all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])
all_playtest_data = pd.concat([playtest6_data])
all_playtest_data = all_playtest_data.combine_first(pd.io.json.json_normalize(all_playtest_data.data.apply(json.loads)))

In [6]:
all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])



In [7]:
all_attempts = all_playtest_data.task_id.value_counts()


In [8]:
complete_puzzles = all_playtest_data.loc[all_playtest_data['type'] == 'ws-puzzle_complete']
successful_attempts = complete_puzzles.task_id.value_counts()

In [9]:
for attempt, value in all_attempts.items():
    success_value = successful_attempts.get(attempt, 0)
    shapes = get_shapes(puzzles.get(attempt))
    print (f"Puzzle {attempt: <25} Attempts {value: < 5} / Success {success_value: < 5} : {round(((success_value/value) * 100), 1): < 5}% Shapes: {shapes}")

Puzzle One Cube                  Attempts  50   / Success  10   :  20.0% Shapes: [1]
Puzzle Match Silhouettes         Attempts  48   / Success  10   :  20.8% Shapes: [1, 2]
Puzzle Rotate a Pyramid          Attempts  42   / Success  10   :  23.8% Shapes: [2]
Puzzle Design a Puzzle           Attempts  40   / Success  0    :  0.0 % Shapes: []
Puzzle Pyramids are Strange      Attempts  36   / Success  6    :  16.7% Shapes: [2, 2, 2, 2]
Puzzle Square Cross-Sections     Attempts  36   / Success  8    :  22.2% Shapes: [3, 4, 2]
Puzzle Separated Cubes           Attempts  36   / Success  10   :  27.8% Shapes: [1, 1]
Puzzle Removing Objects          Attempts  35   / Success  9    :  25.7% Shapes: [2, 5]
Puzzle Shape Limits              Attempts  33   / Success  6    :  18.2% Shapes: [1]
Puzzle Stretch a Ramp            Attempts  32   / Success  9    :  28.1% Shapes: [3]
Puzzle Pi Henge                  Attempts  32   / Success  7    :  21.9% Shapes: [1, 1, 1, 1, 1]
Puzzle Cubes Obscure Spheres  

In [10]:
created_shapes = all_playtest_data.loc[all_playtest_data['type'] == 'ws-create_shape']
print (created_shapes)
#shapeType and 


       actionIndex  cameraAngle  changeMode  color conditions correct  \
23     6.0         NaN           1.0         1.0    NaN        NaN      
37     3.0         NaN           1.0         1.0    NaN        NaN      
62     3.0         NaN           1.0         1.0    NaN        NaN      
92     3.0         NaN           1.0         1.0    NaN        NaN      
108    19.0        NaN           1.0         1.0    NaN        NaN      
112    23.0        NaN           1.0         1.0    NaN        NaN      
150    3.0         NaN           1.0         1.0    NaN        NaN      
151    3.0         NaN           1.0         1.0    NaN        NaN      
171    3.0         NaN           1.0         1.0    NaN        NaN      
172    4.0         NaN           1.0         1.0    NaN        NaN      
174    3.0         NaN           1.0         1.0    NaN        NaN      
189    3.0         NaN           1.0         1.0    NaN        NaN      
208    4.0         NaN           1.0         1.0   

In [11]:
all_playtest_data.type.value_counts()

ws-mode_change                1520
ws-rotate_shape               1202
ws-rotate_view                1088
ws-scale_shape                1004
ws-move_shape                 958 
ws-select_shape               927 
ws-deselect_shape             832 
ws-create_shape               649 
ws-puzzle_started             267 
ws-start_level                245 
ws-check_solution             243 
ws-click_nothing              237 
ws-exit_to_menu               200 
ws-snapshot                   194 
ws-delete_shape               158 
ws-puzzle_complete            143 
ws-undo_action                56  
ws-toggle_snapshot_display    50  
ws-login_user                 22  
ws-restart_puzzle             22  
ws-start_game                 22  
ws-select_shape_add           20  
ws-paint                      14  
ws-palette_change             12  
ws-toggle_paint_display       12  
ws-redo_action                3   
start_game                    1   
Name: type, dtype: int64

In [12]:
all_playtest_data = all_playtest_data.sort_values('time')

In [13]:
data_by_user = all_playtest_data.groupby('session_id').agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'n_events',
                                                                                              'type':'n_different_events'})

data_by_user

,session_id,n_events,n_different_events
0,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,906,22
1,4kocnrzlajnjs74jvmmfqc2w4cx1yupv-Playtest6,1151,24
2,7kd277wea4qj9han9ktqgvtynze7vw16-Playtest6,1154,21
3,c165fzstiysqgc7fb9ugb4x2y17cxsfn-Playtest6,40,9
4,c8n052s1419zhv42bwjyf5tzzb2awj0f-Playtest6,907,23
5,d621ue1gezxve7irtlkwjkt7fo3ge80i-Playtest6,905,23
6,d7fnkcuk3q7hkef890xoddf0af7d0ztb-Playtest6,488,23
7,gllbl7niakw6fnoh7tj32xk42y95jmz9-Playtest6,971,22
8,m05apg7rwfugbgznw6pinp7cdkcs03cw-Playtest6,874,21
9,m1q3lj904h4fbk8u4vt6s0swnri3ai4i-Playtest6,909,20


In [14]:
data_by_user['active_time'] = np.nan
data_by_user['avg_dt'] = np.nan
data_by_user['std_dt'] = np.nan

In [15]:
data_by_user.index = data_by_user['session_id'].values

In [16]:
for user in data_by_user['session_id'].unique():
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
   
    # Computing active time
    previousEvent = None
    theresHoldActivity = 15 # np.percentile(allDifferences, 98) is 10 seconds
    activeTime = []
    
    for enum, event in user_events.iterrows():
        
        # If it is the first event
        if(previousEvent is None):
            previousEvent = event
            continue
        
        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
        if(~(delta_seconds > theresHoldActivity)):
            activeTime.append(delta_seconds)
        
        previousEvent = event
        
    data_by_user.at[user, 'active_time'] = round(np.sum(activeTime)/60,2)
    data_by_user.at[user, 'avg_dt'] = round(np.mean(activeTime)/60,2)
    data_by_user.at[user, 'std_dt'] = round(np.std(activeTime)/60,2)

In [17]:
data_by_user

,session_id,n_events,n_different_events,active_time,avg_dt,std_dt
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,906,22,94.25,0.10,1.45
4kocnrzlajnjs74jvmmfqc2w4cx1yupv-Playtest6,4kocnrzlajnjs74jvmmfqc2w4cx1yupv-Playtest6,1151,24,110.74,0.10,1.27
7kd277wea4qj9han9ktqgvtynze7vw16-Playtest6,7kd277wea4qj9han9ktqgvtynze7vw16-Playtest6,1154,21,84.38,0.07,1.26
c165fzstiysqgc7fb9ugb4x2y17cxsfn-Playtest6,c165fzstiysqgc7fb9ugb4x2y17cxsfn-Playtest6,40,9,15.41,0.40,0.93
c8n052s1419zhv42bwjyf5tzzb2awj0f-Playtest6,c8n052s1419zhv42bwjyf5tzzb2awj0f-Playtest6,907,23,97.07,0.11,1.67
d621ue1gezxve7irtlkwjkt7fo3ge80i-Playtest6,d621ue1gezxve7irtlkwjkt7fo3ge80i-Playtest6,905,23,93.71,0.10,1.66
d7fnkcuk3q7hkef890xoddf0af7d0ztb-Playtest6,d7fnkcuk3q7hkef890xoddf0af7d0ztb-Playtest6,488,23,106.43,0.22,2.62
gllbl7niakw6fnoh7tj32xk42y95jmz9-Playtest6,gllbl7niakw6fnoh7tj32xk42y95jmz9-Playtest6,971,22,88.75,0.09,1.39
m05apg7rwfugbgznw6pinp7cdkcs03cw-Playtest6,m05apg7rwfugbgznw6pinp7cdkcs03cw-Playtest6,874,21,94.53,0.11,1.64
m1q3lj904h4fbk8u4vt6s0swnri3ai4i-Playtest6,m1q3lj904h4fbk8u4vt6s0swnri3ai4i-Playtest6,909,20,95.60,0.11,1.62
